## Creating an AzureML Environment for Stan
First, we'll create a Docker container by way of an AzureML Environment. 

For this, I've written a [Dockerfile](../Dockerfile) that downloads and installs Stan. This is based on the IntelMPI image published by the Azure ML team. (You can find the Dockerfile for that image [here](https://github.com/Azure/AzureML-Containers/blob/master/base/cpu/intelmpi2018.3-ubuntu16.04/Dockerfile).)

In [ ]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()

For this example, we've built the Dockerfile and pushed it to the Azure Container Registry that was created with AML. It is possible to pass the Dockerfile text as a parameter to the Environment, but as of January 27th, 2020 this is not supported in ML Pipelines.


```python
# Read our Dockerfile into a Python string object. 
with open('../Dockerfile', 'r') as file_obj:
    docker_file = file_obj.read()
```

```python
env.docker.base_image = None
env.docker.base_dockerfile = docker_file
```

In [ ]:
# Instatiate an Environment object from AML. We've named it 'stan-intelmpi'
env = Environment('stan-intelmpi')

# Next, set the Docker settings for the environment - first setting Docker to enabled, then setting the 
# base_dockerfile property to the Docker string we published before.
# Note - we set the base_image property to None - as base_image and base_dockerfile are mutually exclusive
env.docker.enabled = True
env.docker.base_image = "amldemocr.azurecr.io/cmdstan:intelmpi2018.3-ubuntu16.04-stan2.21"
env.docker.base_dockerfile = None

# Next, set the Python settings. This will set the pip or conda packages that need to be installed in the container.
env.python.conda_dependencies.set_python_version('3.7.6')
env.python.conda_dependencies.add_pip_package('azureml-defaults')
env.python.conda_dependencies.add_pip_package('cmdstanpy==0.8.0')
env.python.conda_dependencies.add_pip_package('mpi4py==3.0.3')
env.python.conda_dependencies.add_pip_package('azureml-dataprep[fuse,pandas]')

# Finally, register the environment.
env = env.register(ws)

In [ ]:
# This is NOT required, but you can prebuild the Docker container by running the env.build step.
# If you choose not to do this, the container will be build the first time you submit the model for building

env_build = env.build(ws)
env_build.wait_for_completion(show_output=True)